For Accessing Google Drive Files

In [1]:
import os,sys
from google.colab import drive
nb_path = '/content/notebooks'
drive.mount('/content/gdrive/')
os.symlink('/content/gdrive/My Drive/Colab Note Notebooks',nb_path)
sys.path.insert(0,nb_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
!pip install --target=$nb_path nltk

Import Neccessary Packages

In [14]:
import tensorflow, nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json, pickle
import random, numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

Load Json Data Files

In [3]:
words, documents, classes = [], [], []
ignore_words = ['!','?']
data_file = open('/content/gdrive/My Drive/Dataset/ChatBot-File/intents.json').read()
#print(data_file)
intents = json.loads(data_file)
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

PreProcess of Data

In [4]:
nltk.download('punkt')
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
nltk.download('wordnet')
words = [ lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print("Documents :")
print(len(documents),documents)
print('Classes :')
print(len(classes),classes)
print('Words ')
print(len(words),words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Documents :
47 [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'A

In [ ]:
pickle.dump(words,open('/content/gdrive/My Drive/Dataset/ChatBot-File/Words.pkl','wb'))
pickle.dump(classes,open('/content/gdrive/My Drive/Dataset/ChatBot-File/Classes.pkl','wb'))

Create Training and Testing Data

In [10]:
training = []
output_empty = [0]*len(classes)
for doc in documents:
  pattern_words = doc[0]
  pattern_words = [ lemmatizer.lemmatize(w.lower()) for w in pattern_words ]
  bag = []
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])]=1

  training.append([bag,output_row])
#print(training)
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

Model Creation

In [13]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(len(train_x[0]),) ))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd = SGD(lr=0.01,momentum=0.9,decay=1e-6,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, verbose=1, batch_size=5)
model.save("/content/gdrive/My Drive/Dataset/ChatBot-File/Chatbot_model.h5",hist)

Epoch 1/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1730 - accuracy: 0.1489
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1009 - accuracy: 0.1915
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0562 - accuracy: 0.2340
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9524 - accuracy: 0.2766
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9303 - accuracy: 0.3404
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7135 - accuracy: 0.4255
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5613 - accuracy: 0.5532
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.4823 - accuracy: 0.6170
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3834 - accuracy: 0.6383
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.1872 - accuracy: 0.5745
Epoch 11/

Predict the Responses for Tag

In [17]:
model = load_model('Chatbot_model.h5')
intents = json.loads(open('/content/gdrive/My Drive/Dataset/ChatBot-File/intents.json').read())
words = pickle.load(open('/content/gdrive/My Drive/Dataset/ChatBot-File/Words.pkl','rb'))
classes = pickle.load(open('/content/gdrive/My Drive/Dataset/ChatBot-File/Classes.pkl','rb'))

In [18]:

def Clean_sentense(sentense):
  '''we will need to provide input in the same way as we did while training'''
  sentense_words = nltk.word_tokenize(sentense)
  sentennse_words = [ lemmatizer.lemmatize(w.lower()) for w in sentense_words]
  return sentense_words

def bow(sentense,words,show_details=True):
  '''return bag of words array: 0 or 1 for each word in the bag that exists in the sentence'''
  sentense_words = Clean_sentense(sentense)
  bag = [0]*len(words)
  for s in sentense_words:
    for i,w in enumerate(words):
      if w is s:
        bag[i]=1
        if show_details:
          print('Found in bag ',w)
  return np.array(bag)

def Predict_class(sentense,model):
  '''Predicting words belong to which intent or which class'''
  p = bow(sentense,words,show_details=False)
  res = model.predict(np.array([p]))[0]
  ERROR_THRESHOLD = 0.25
  results = [ [i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  results.sort(key = lambda x:x[1], reverse=True)
  return_list = []
  for i in results:
    return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
  return return_list


In [26]:
def geResponse(ints,intents_json):
  '''After predicting the class, we will get a random response from the list of intents.'''
  list_of_intents = intents_json['intents']
  tag = ints[0]['intent']
  for i in list_of_intents:
    if i['tag']==tag:
      result = random.choice(i['responses'])
      break
  return result

def chatbot_response(text):
  ints = Predict_class(text,model)
  res = getResponse(ints,intents)
  return res

Creating GUI for ChatBot

In [ ]:
import tkinter as tk
#from tkinter import *

def send():
  msg = EntryBox.get('1.0','end-1c').strip()
  EntryBox.Delete('0.0',END)
  if msg!="":
    ChatLog.config(state='Normal')
    ChatLog.Insert(END,"You: "+ msg + '\n\n')
    ChatLog.config(foreground='#442265',font=("Verdana", 12 ))

    res = chatbot_response(msg)
    ChatLog.Insert(END,"Bot: "+ res + '\n\n')

    ChatLog.config(state=DISABLED)
    ChatLog.yview(END)

base = tk.Tk()
base.title('ChatBot')
base.geomentry("400x500")
base.resizable(width=FALSE,height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg='blue', height="8", width="50", font="Arial")
ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()